In [ ]:
class HybridRecommender:
    """ gets the two previous methods and multiplies the 
    scores to get an overall score"""


    MODEL_NAME = 'Hybrid_1'


    def __init__(self, content_based_model,
                 collab_filtering_model,
                 projects):
        
        self.content_based_model = content_based_model
        self.collab_filtering_model = collab_filtering_model
        self.projects = projects

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_projects(self, donor_id,
                           projects_to_ignore=[],
                           top_n = 10):
        """get the results of each and combine them"""


        # results from content based
        content_b_df = self.content_based_model\
        .recommend_projects(donor_id, 
                            projects_to_ignore,
                            top_n = 1000)\
                            .rename(columns={'recommStrength':'ContentBasedStrength'})

        # resutls of collaborate filtering

        collab_filtering_df = self.collab_filtering_model\
        .recommend_projects(donor_id, 
                            projects_to_ignore,
                            top_n = 1000)\
                            .rename(columns={'recommStrength':'CollabFilteringStrength'})


        recomm_df = pd.merge(content_b_df,
                             collab_filtering_df,
                             how='inner',
                             on = 'Project ID')
        
        # multiplying two scores
        recomm_df['HybridStrength'] =\
        recomm_df['CollabFilteringStrength']*recomm_df['ContentBasedStrength']
        
        # sorting by the hybrid score
        recomm_df = recomm_df.sort_values(by='HybridStrength',
                                          ascending=False).head(top_n)
        # merging all the data with the projects dataset
        recomm_df = pd.merge(recomm_df,
                             self.projects, 
                             how='left',
                             on='Project ID')[['HybridStrength', 
                                               'Project ID', 'Project Title', 
                                               'Project Essay',
                                               'ContentBasedStrength',
                                               'CollabFilteringStrength']]

        return recomm_df